In [2]:
from sklearn.metrics import accuracy_score
import os
import sys
sys.path.append("../")
import numpy as np
from datetime import datetime
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from settings import PROCESSED_DATA_DIR, MODELS_DIR
from src.features.helpers.training import make_lstm_model, get_early_stopping_definition, plot_training_validation_loss


ML_MODEL = 'test_visualisation'

print(f'{datetime.now().strftime("%H:%M:%S")} Start data loading...')
x_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'x_train.npy'), allow_pickle=True)
y_train = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_train.npy'), allow_pickle=True)

x_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'x_val.npy'), allow_pickle=True)
y_val = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_val.npy'), allow_pickle=True)

x_test = np.load(os.path.join(PROCESSED_DATA_DIR, 'x_test.npy'), allow_pickle=True)
y_test = np.load(os.path.join(PROCESSED_DATA_DIR, 'y_test.npy'), allow_pickle=True)

print(f'{datetime.now().strftime("%H:%M:%S")} End data loading...')

# num_positives = y_train_num.count(1)
num_positives = np.count_nonzero(y_train == 1)

# num_negatives = y_train.count(0)
num_negatives = np.count_nonzero(y_train == 0)

total = len(y_train)

print(
    f'Examples:\n    Total: {total}\n    Positive: {num_positives} ({round(100 * num_positives / total, 2)}% of total)\n')

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = 1
weight_for_1 = int(num_negatives / num_positives) if num_positives != 0 else 0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

initial_bias = np.log([num_positives / num_negatives])

# ### Input is sensitive (make sure that big numbers are not causing loss issue (could be gameIdplayId column))
# ### https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network

EPOCHS = 20
BATCH_SIZE = 32
model = KerasClassifier(
    build_fn=make_lstm_model,
    input_dim=(x_train.shape[1], x_train.shape[2]),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

print(f'{datetime.now().strftime("%H:%M:%S")} Start training...')

history = model.fit(
    x_train,
    y_train,
    class_weight=class_weight,
    validation_data=(x_val, y_val),
    callbacks=[get_early_stopping_definition(mode='min')]
)

print(f'{datetime.now().strftime("%H:%M:%S")} End training...')
model.model.summary()

print("Saving model...")
model.model.save(os.path.join(MODELS_DIR, f"{ML_MODEL}_rnn.h5"))

09:19:38 Start data loading...
09:19:38 End data loading...
Examples:
    Total: 11
    Positive: 0 (0.0% of total)

Weight for class 0: 1.00
Weight for class 1: 0.00
09:19:38 Start training...


<ipython-input-2-ffe823832a04>:47: RuntimeWarning: divide by zero encountered in log
  initial_bias = np.log([num_positives / num_negatives])


Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 00004: early stopping
09:19:55 End training...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 84, 178)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 84, 256) 

In [31]:
x = x_train[:50]
orig_out = model.model.predict(x)

In [32]:
orig_out.shape

(11, 1)

In [35]:
x_train[0].shape[1]

178

In [37]:
for i in range(x_train[0].shape[1]):  # iterate over the three features
    new_x = x.copy()
    perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
    new_x[:, :, i] = new_x[:, :, i] + perturbation
    perturbed_out = model.model.predict(new_x)
    effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
    print(f'Variable {i+1}, perturbation effect: {effect:.4f}')

Variable 1, perturbation effect: 0.0321
Variable 2, perturbation effect: 0.0321
Variable 3, perturbation effect: 0.0318
Variable 4, perturbation effect: 0.0320
Variable 5, perturbation effect: 0.0321
Variable 6, perturbation effect: 0.0321
Variable 7, perturbation effect: 0.0320
Variable 8, perturbation effect: 0.0318
Variable 9, perturbation effect: 0.0319
Variable 10, perturbation effect: 0.0319
Variable 11, perturbation effect: 0.0320
Variable 12, perturbation effect: 0.0319
Variable 13, perturbation effect: 0.0320
Variable 14, perturbation effect: 0.0319
Variable 15, perturbation effect: 0.0320
Variable 16, perturbation effect: 0.0319
Variable 17, perturbation effect: 0.0317
Variable 18, perturbation effect: 0.0316
Variable 19, perturbation effect: 0.0319
Variable 20, perturbation effect: 0.0316
Variable 21, perturbation effect: 0.0319
Variable 22, perturbation effect: 0.0319
Variable 23, perturbation effect: 0.0323
Variable 24, perturbation effect: 0.0320
Variable 25, perturbation

## Conclusion: no feature is important enough -> this needs to be tested on real model